In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
import os
from datetime import datetime
from IPython.display import display, Markdown
from pathvalidate import sanitize_filename

import utils

from prompts import Job_Post, Resume_Builder

In [108]:
## Inputs
my_files_dir = "my_applications"  # location for all job and resume files
job_file = "job.txt"  # filename with job post text. The entire job post can be pasted in this file, as is.
raw_resume_file = "resume_raw.yaml"  # filename for raw resume yaml. See example in repo for instructions.

# Model for matching resume to job post (except extraction chains)
# gpt-4 is not publicly available yet and can be 20-30 times costlier than the default model gpt-3.5-turbo.
# Simple extraction chains are hardcoded to use the cheaper gpt-3.5 model.
openai_model_name = "gpt-4"
openai_model_name = "gpt-3.5-turbo"

# temperature lower than 1 is preferred. temperature=0 returns deterministic output
temperature = 0.5

llm_kwargs = dict(
    model_name=openai_model_name,
    model_kwargs=dict(top_p=0.6, frequency_penalty=0.1),
)

In [109]:
# Step 1 - Read and parse job posting
job_post = Job_Post(
    utils.read_jobfile(os.path.join(my_files_dir, job_file)),
)
parsed_job = job_post.parse_job_post(verbose=False)

company_name = parsed_job["company"]
job_title = parsed_job["job_title"]
today_date = datetime.today().strftime("%Y%m%d")
job_filename = os.path.join(
    my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}")
)
print(f"#filename: {job_filename}.job\n")
utils.write_yaml(parsed_job, filename=f"{job_filename}.job")

#filename: my_applications\20230915__NOIR__VueJS Developer.job

company: NOIR
job_title: VueJS Developer
team:
job_summary: VueJS Developer position at a global record label in London, working
  on critical projects for their new online music shopping channel.
salary: Â£80,000 - Â£100,000 per annum
duties:
- Developing web applications using VueJS, HTML5, CSS3, TypeScript, GIT Version Control,
  JavaScript, SASS, jQuery, React, and AJAX
- Working on projects that are essential to the launch of the online music shopping
  channel
- Collaborating with the team to deliver high-quality code and user experiences
- Participating in code reviews and providing constructive feedback
- Keeping up to date with the latest web development trends and technologies
qualifications:
- Experience in VueJS, HTML5, CSS3, TypeScript, GIT Version Control, JavaScript, SASS,
  jQuery, React, and AJAX
- Ability to learn and adapt to new technologies
- Strong problem-solving and analytical skills
- Excellent com

In [110]:
# Step 2 - read raw resume and create Resume builder object
my_resume = Resume_Builder(
    resume=utils.read_yaml(filename=os.path.join(my_files_dir, raw_resume_file)),
    parsed_job=parsed_job,
    llm_kwargs=llm_kwargs,
)
projects = my_resume.projects_raw
experiences = my_resume.experiences_raw
skills = my_resume.skills_raw

In [113]:
# Step 3 - Rephrase unedited experiences
experiences = my_resume.rewrite_unedited_experiences(verbose=False)
utils.write_yaml(dict(experiences=experiences))

experiences:
- company: SHANGHAI FRIENDESS ELECTRONICS TECHNOLOGY CO., LTD. Shanghai, China
  titles:
  - name: Web Front-end Development Engineer - Software R&D Department
    startdate: 2021.07
    enddate: 2022.08
    # highlights:
    # the unedited tag in experiences can be used to automatically have the LLM rewrite it as highlights
  highlights:
  - Developed web-based features using VueJS and Typescript for customized PDF and
    Excel reports with 200,000+ unique users per month, demonstrating ability to develop
    web applications using VueJS, TypeScript, and deliver high-quality code and user
    experiences.
  - Created an automatic 3D house layout model generation system using VueJS and BabylonJS,
    showcasing ability to work on projects essential to the launch of online channels
    and deliver high-quality code and user experiences.
  - Full-stack developed a material calculation system using VueJS and Flask, reducing
    production time by up to 30% for users, demonst

In [105]:
experiences

[{'company': 'SHANGHAI FRIENDESS ELECTRONICS TECHNOLOGY CO., LTD. Shanghai, China', 'titles': [{'name': 'Web Front-end Development Engineer - Software R&D Department', 'startdate': 2021.07, 'enddate': 2022.08}], 'unedited': ['Worked with agile team to develop web-based features using VueJS and Typescript for customized PDF and Excel reports with 200,000+ unique users per month.', 'Developed automatic 3D house layout model generation system in a pre-research using VueJS and BabylonJS to ensure a user-friendly experience.', 'Full-stack developed material calculation system using VueJS and Flask, which reduced production time by up to 30% for users.']}, {'company': 'TUQI CONSTRUCTION TECHONOLOGY CO., LTD. Shanghai, China', 'titles': [{'name': 'Web Front-end Engineer - Development Department', 'startdate': 2021.03, 'enddate': 2021.05}], 'unedited': ['Automated verification, collection, and cleaning of offline data using Node.js, jQuery, and LayUI. Enhanced data submission efficiency across

In [112]:
# Step 4 - Rephrase projects
my_resume.experiences = experiences

projects = my_resume.rewrite_section(section=my_resume.projects_raw, verbose=False)
utils.write_yaml(dict(projects=projects))

TypeError: sequence item 0: expected str instance, CommentedMap found

In [104]:
section = my_resume.projects_raw
input_variables = ['technical_skills', 'duties', 'non_technical_skills', 'section', 'qualifications']

In [103]:
def format_prompt_inputs_as_strings(prompt_inputs: list[str], **kwargs):
    """Convert values to string for all keys in kwargs matching list in prompt inputs"""
    return {
        k: format_list_as_string(v) for k, v in kwargs.items() if k in prompt_inputs
    }

In [ ]:
format_prompt_inputs_as_strings(input_variables, )

In [93]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed
edits = """ """

edits = edits.strip()
if edits:
    edits2 = utils.read_yaml(edits)
    if "experiences" in edits2:
        experiences = edits2["experiences"]
    if "projects" in edits2:
        projects = edits2["projects"]
    if "skills" in edits2:
        skills = edits2["skills"]
    if "summary" in edits2:
        summary = edits2["summary"]

In [94]:
# Step 5 - Extract skills
# This will match the required skills from the job post with your resume sections
# Outputs a combined list of skills extracted from the job post and included in the raw resume
my_resume.experiences = experiences
my_resume.projects = projects

skills = my_resume.extract_matched_skills(verbose=False)
utils.write_yaml(dict(skills=skills))

TypeError: sequence item 0: expected str instance, CommentedMap found

In [48]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed
edits = """ """

edits = edits.strip()
if edits:
    edits3 = utils.read_yaml(edits)
    if "experiences" in edits3:
        experiences = edits3["experiences"]
    if "projects" in edits3:
        projects = edits3["projects"]
    if "skills" in edits3:
        skills = edits3["skills"]
    if "summary" in edits3:
        summary = edits3["summary"]

In [49]:
# Step 6 - Create a resume summary
my_resume.experiences = experiences
my_resume.skills = skills
my_resume.projects = projects

summary = my_resume.write_summary(
    verbose=True,
)
utils.write_yaml(dict(summary=summary))



> Entering new  chain...
Prompt after formatting:
System: You are an expert technical writer. Your goal is to strictly follow all the provided <Steps> and meet all the given <Criteria>.
Human: <Job Posting>
Amazon Data Services Ireland Limited
Amazon's Enterprise Network team is looking to hire Network Engineers to help scale and automate one of the world's largest and complex networks. The role involves deploying network infrastructure, scaling and automating the network across global datacenters, and developing tooling to increase scaling velocity and quality.
Human: <Resume>
Education:
- Doctorate in Fast Computing
- Master's in Arithmetic
- Bachelor's in Mathematics

Experience:
- Highlight 1: Authored and published over 10 books on diverse subjects such as math, astrology, memory, and puzzles, demonstrating exceptional research skills, effective communication, and expertise in knowledge dissemination.
- Highlight 2: Successfully toured globally, showcasing exceptional computing 

In [50]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed.
edits = """ """

edits = edits.strip()
if edits:
    edits4 = utils.read_yaml(edits)
    if "experiences" in edits4:
        experiences = edits4["experiences"]
    if "projects" in edits4:
        projects = edits4["projects"]
    if "skills" in edits4:
        skills = edits4["skills"]
    if "summary" in edits4:
        summary = edits4["summary"]

In [51]:
# Step 7 - Generate final resume yaml for review
my_resume.summary = summary
my_resume.experiences = experiences
my_resume.projects = projects
my_resume.skills = skills

today_date = datetime.today().strftime("%Y%m%d")
resume_filename = os.path.join(
    my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}")
)
resume_final = my_resume.finalize()
print(f"#filename: {resume_filename}.yaml\n")
utils.write_yaml(resume_final, filename=f"{resume_filename}.yaml")

#filename: my_applications\20230914__Amazon Data Services Ireland Limited__Network Engineer.yaml

basic:
  name: Shakuntala Devi
  address:
  - Bangalore, Karnataka, India
  contact:
    email: calculate@shakuntla.devi
    phone: 314 159 2653
  websites:
  # optionally add icons to websites from fontawesome - http://mirrors.ctan.org/fonts/fontawesome/doc/fontawesome.pdf
  - text: guinness.com/fastest-computer
    url: https://www.guinnessworldrecords.com/world-records/67741-human-computation
    icon: beer

# summary will be overwritten by the LLM. Leave blank.
summary: Based on the <Plan> and <Additional Steps>, the Professional Summary has
  been created.
education:
- school: Self-educated
  degrees:
  - names:
    - Doctorate in Fast Computing
    - Master's in Arithmetic
    startdate: 1949
    enddate: 1956
    gpa: 4.0
  - names:
    - Bachelor's in Mathematics
    startdate: 1937
    enddate: 1939
  achievements:
  - Taught card tricks to teachers
  - Gold medal in Arithmetics



In [52]:
# Step 8 - Identify resume improvements
# A previously generated resume can also be used here by manually providing resume_filename. Requires the associated parsed job file.
final_resume = Resume_Builder(
    resume=utils.read_yaml(filename=f"{resume_filename}.yaml"),
    parsed_job=utils.read_yaml(filename=f"{resume_filename}.job"),
    is_final=True,
    llm_kwargs=llm_kwargs,
)
improvements = final_resume.suggest_improvements(verbose=True)
utils.write_yaml(dict(improvements=improvements))



> Entering new  chain...
Prompt after formatting:
System: You are an expert critic. Your goal is to strictly follow all the provided <Steps> and meet all the given <Criteria>.
Human: <Job Posting>
The ideal candidate is able to perform the following duties:
- Deploy network infrastructure across all network fabrics that support the Amazon Retail Service and AWS services
- Scale and automate the network across global datacenters
- Identify opportunities and develop tooling to increase scaling velocity and quality
- Create simple, repeatable processes and improve upon existing processes
- Work closely with internal customers on designs/solutions and bring them from concept to production
- Create and update network standards and ensure deployment to these standards
- Work closely with teams to define platforms and frameworks for unprecedented volume scaling

The ideal candidate has the following qualifications:
- Bachelor's Degree in a technology-related field
- 5+ years professional ex

improvements:
- section: spelling and grammar
  improvements:
  - '- In the Experiences section, ensure proper grammar and sentence structure in
    the descriptions.'
  - '- In the Projects section, review sentence structure and clarity.'
  - '- In the Education section, double-check for any spelling errors or typos.'
  - '- In the Skills section, ensure consistent capitalization and proper punctuation.'
- section: experience
  improvements:
  - '- Improve: Provide relevant experiences related to networking or technology roles
    that demonstrate experience in large-scale networking environments or working
    with network operating systems.'
  - '- How to improve: Remove or rephrase experiences that are not directly related
    to the job requirements and add experiences that showcase relevant skills and
    knowledge.'
- section: projects
  improvements:
  - '- Improve: Include projects or achievements that highlight experience or knowledge
    in networking, programming, or relate

In [54]:
# Step 9 - Generate pdf from yaml
# Most common errors during pdf generation occur due to special characters. Escape them with backslashes in the yaml, e.g. $ -> \$
pdf_file = utils.generate_pdf(yaml_file=f"{resume_filename}.yaml")
# display(Markdown((f"[{pdf_file}](<{pdf_file}>)")))

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [72]:
utils.generate_new_tex(yaml_file=f"{resume_filename}.yaml")